In [ ]:
# Import library
import os
import pandas as pd
import numpy as np
import warnings
import time
warnings.filterwarnings("ignore")

# export data

**Export site data related to age in normal sample**

In [ ]:
meta = pd.read_csv('../../train_data/meta.csv')
merge = pd.read_csv('../../train_data/merge_Blood_withchr.csv',index_col=0)
merge.index = [str(i) for i in merge.index.tolist()]
df = pd.read_csv('../../train_data/bootstrap_Blood_0.2.csv',index_col=0)
merge = merge.loc[df.columns.tolist()[:-1]]

# Match age information to merge data
merge_data = pd.merge(merge.T,meta[["SRA","age","disease state"]],left_index=True,right_on=["SRA"])
merge_data = merge_data.set_index("SRA")

merge_data2 = merge_data.loc[merge_data['disease state'] == 'Normal',:]

merge_check = merge_data2.iloc[:,0:merge.shape[0]].isnull()

# Calculate correlation between single cpgsite and age
merge['corr'] = [np.corrcoef(merge_data2.iloc[(~merge_check.iloc[:,i]).tolist(),i].tolist(), merge_data2.iloc[(~merge_check.iloc[:,i]).tolist(),-2].tolist())[0,1] for i in range(merge_data2.shape[1]-2)]

# Sort corr column from high to low
merge1 = merge.sort_values(by='corr',ascending=False)
# Delete rows with corr as NA
merge2 = merge1.loc[~merge1['corr'].isna(),:]
# save data
nums = [500,1000,2000,3000]
for num in nums:
    merge2.iloc[0:num,0:3].to_csv(f'normal_positive_{num}.csv')
    merge2.iloc[-num:,0:3].to_csv(f'normal_negative_{num}.csv')

**Export site data related to age in disease sample**

In [ ]:
meta = pd.read_csv('../../train_data/meta.csv')
merge = pd.read_csv('../../train_data/merge_Blood_withchr.csv',index_col=0)
merge.index = [str(i) for i in merge.index.tolist()]
df = pd.read_csv('../../train_data/bootstrap_Blood_0.2.csv',index_col=0)
merge = merge.loc[df.columns.tolist()[:-1]]

# Match age information to merge data
merge_data = pd.merge(merge.T,meta[["SRA","age","disease state"]],left_index=True,right_on=["SRA"])
merge_data = merge_data.set_index("SRA")

merge_data2 = merge_data.loc[merge_data['disease state'] != 'Normal',:]

merge_check = merge_data2.iloc[:,0:merge.shape[0]].isnull()

# Calculate correlation between single cpgsite and age
merge['corr'] = [np.corrcoef(merge_data2.iloc[(~merge_check.iloc[:,i]).tolist(),i].tolist(), merge_data2.iloc[(~merge_check.iloc[:,i]).tolist(),-2].tolist())[0,1] for i in range(merge_data2.shape[1]-2)]

# Sort corr column from high to low
merge1 = merge.sort_values(by='corr',ascending=False)
# Delete rows with corr as NA
merge2 = merge1.loc[~merge1['corr'].isna(),:]
# save data
nums = [500,1000,2000,3000]
for num in nums:
    merge2.iloc[0:num,0:3].to_csv(f'disease_positive_{num}.csv')
    merge2.iloc[-num:,0:3].to_csv(f'disease_negative_{num}.csv')

# GREAT analysis

Enter the above data into [GREAT](http://great.stanford.edu/public/html/)
* Select version **GREAT version 4.0.4**;
* Select **Human: GRCh37 (UCSC hg19, Feb. 2009)** for Species Assembly;
* Select **BED data** for Test regions, and paste the chromosome and start and end data into it;
* Click **Submit** to download the corresponding results.